# Pharmacy Demand Forecasting - Model Training

This notebook builds and trains machine learning models for predicting pharmacy order quantities.

## Objectives:
1. Load and preprocess the data
2. Feature engineering and encoding
3. Train baseline models (Linear Regression, Random Forest, XGBoost)
4. Evaluate models using RMSE/MAE
5. Save the best model for deployment

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import joblib
import sys
from pathlib import Path

# Add app directory to path
sys.path.append('../app')
from utils import preprocess_sales_features, save_model, parse_order_scheme

plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Data Loading and Preprocessing

In [ ]:
# Load processed data from EDA
try:
    df = pd.read_csv('../data/processed_data.csv')
    print(f"Loaded {len(df)} records")
except FileNotFoundError:
    print("Processed data not found. Creating sample data...")
    # Create sample data
    np.random.seed(42)
    n_samples = 1000
    df = pd.DataFrame({
        'L7': np.random.randint(0, 50, n_samples),
        'L15': np.random.randint(0, 100, n_samples),
        'L30': np.random.randint(0, 200, n_samples),
        'L60': np.random.randint(0, 400, n_samples),
        'Order': np.random.choice(['12', '9+1', '24', '6+1', '18', '15'], n_samples)
    })
    
    # Parse orders
    order_parsed = df['Order'].apply(parse_order_scheme)
    df['Base_Quantity'] = [x[0] for x in order_parsed]
    df['Scheme_Type'] = [x[1] for x in order_parsed]
    
    df = preprocess_sales_features(df)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

In [ ]:
# Define features and target
feature_cols = [col for col in df.columns if col.startswith('L') or 
                col in ['sales_trend_7_15', 'sales_trend_15_30', 'avg_sales_velocity', 
                       'max_sales_period', 'sales_volatility']]

target_col = 'Base_Quantity'

print(f"Feature columns: {feature_cols}")
print(f"Target column: {target_col}")

# Remove rows with missing target
df_clean = df.dropna(subset=[target_col])
print(f"Clean dataset shape: {df_clean.shape}")

## 2. Feature Engineering

In [ ]:
# Prepare features
X = df_clean[feature_cols].fillna(0)
y = df_clean[target_col]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target statistics:")
print(y.describe())

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 3. Model Training

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled features for Linear Regression, original for tree-based
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"RMSE: {rmse:.3f}")
    print(f"MAE: {mae:.3f}")
    print(f"R²: {r2:.3f}")

## 4. Model Evaluation

In [ ]:
# Compare model performance
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[model]['rmse'] for model in results.keys()],
    'MAE': [results[model]['mae'] for model in results.keys()],
    'R²': [results[model]['r2'] for model in results.keys()]
})

print("Model Comparison:")
print(comparison_df)

# Find best model
best_model_name = comparison_df.loc[comparison_df['RMSE'].idxmin(), 'Model']
best_model = results[best_model_name]['model']
print(f"\nBest model: {best_model_name}")

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Model comparison
comparison_df.set_index('Model')[['RMSE', 'MAE']].plot(kind='bar', ax=axes[0, 0])
axes[0, 0].set_title('Model Performance Comparison')
axes[0, 0].set_ylabel('Error')
axes[0, 0].tick_params(axis='x', rotation=45)

# R² comparison
comparison_df.set_index('Model')['R²'].plot(kind='bar', ax=axes[0, 1], color='green')
axes[0, 1].set_title('R² Score Comparison')
axes[0, 1].set_ylabel('R² Score')
axes[0, 1].tick_params(axis='x', rotation=45)

# Actual vs Predicted for best model
best_predictions = results[best_model_name]['predictions']
axes[1, 0].scatter(y_test, best_predictions, alpha=0.6)
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual')
axes[1, 0].set_ylabel('Predicted')
axes[1, 0].set_title(f'Actual vs Predicted - {best_model_name}')

# Residuals plot
residuals = y_test - best_predictions
axes[1, 1].scatter(best_predictions, residuals, alpha=0.6)
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Predicted')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title(f'Residuals Plot - {best_model_name}')

plt.tight_layout()
plt.show()

## 5. Feature Importance

In [ ]:
# Feature importance for tree-based models
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Feature Importance:")
    print(feature_importance)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
    plt.title(f'Top 10 Feature Importance - {best_model_name}')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
elif best_model_name == 'Linear Regression':
    # Coefficients for linear regression
    coefficients = pd.DataFrame({
        'feature': feature_cols,
        'coefficient': best_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("Linear Regression Coefficients:")
    print(coefficients)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=coefficients.head(10), x='coefficient', y='feature')
    plt.title('Top 10 Feature Coefficients - Linear Regression')
    plt.xlabel('Coefficient')
    plt.tight_layout()
    plt.show()

## 6. Model Saving

In [ ]:
# Create models directory
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Save the best model
model_path = models_dir / 'order_predictor.pkl'
save_model(best_model, str(model_path))
print(f"Best model saved to: {model_path}")

# Save scaler if needed
if best_model_name == 'Linear Regression':
    scaler_path = models_dir / 'scaler.pkl'
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to: {scaler_path}")

# Save feature columns
feature_info = {
    'feature_columns': feature_cols,
    'model_type': best_model_name,
    'performance': {
        'rmse': results[best_model_name]['rmse'],
        'mae': results[best_model_name]['mae'],
        'r2': results[best_model_name]['r2']
    }
}

import json
with open(models_dir / 'model_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print("Model training completed successfully!")
print(f"Final model performance:")
print(f"- RMSE: {results[best_model_name]['rmse']:.3f}")
print(f"- MAE: {results[best_model_name]['mae']:.3f}")
print(f"- R²: {results[best_model_name]['r2']:.3f}")